## Implementation of Regression Model using tensorflow dense layers and comparing with Linear regression

#### Import the required python packages

In [0]:
from matplotlib import pyplot as plt
from matplotlib import style
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split


####Assumptions of Linear Regression:

1. There should be linear relationship b/w dependent and independent variables.
2. Target variable(dependent) should be normal distribution -if not transform it to normal distribution using log/exponential/square root etc. functions
3. The independent variables should not be correlated.
4. Error must have constant varience.


#### Read the read from the GIT HUB

In [0]:
dframe = pd.read_csv("https://raw.githubusercontent.com/insaid2018/Term-2/master/CaseStudy/Advertising.csv",index_col=0)
dframe.head()

### Data description
#### Features
TV - spend in dollors on TV advertisement for a sigle product (thousands of dollors)

Radio - spend in dollors on Radio advertisement

newspaper - spend in dollors on Newspaper advertisement

#### Labels
sales - Sales of a single product in the market (in thousands of widgets)

As we are trying to predict the value of the sales, this is a regression problem

Need to do preprocessing on the data before designing a model.

1.   Check for missing values and replace numeric values with mean/median and categorical values with mode.
2.   Replace the categorical values - using label encoding or dummification.
3.   Normalise the data to same scale if needed.



In [0]:
dframe.isna().sum()

In [0]:
dframe_describe = dframe.describe().transpose()
dframe_describe.head()

In [0]:
sns.pairplot(dframe)

#### Observations from above plots:

Amount spend on TV has linear relationship with the sales.

Amount spend on Radio has some linearity.

Amount spend on newspaper has very min linear relationship with the sales.

Dependent variable(sales) is normally distributed.

There is no relationship between independent variables







#### Normalise the Data to the same scale

It is good practice to normalize features that use different scales and ranges. Although the model might converge without feature normalization, it makes training more difficult, and it makes the resulting model dependent on the choice of units used in the input.

Note: Although we intentionally generate these statistics from only the training dataset, these statistics will also be used to normalize the test dataset. We need to do that to project the test dataset into the same distribution that the model has been trained on.

In [0]:
#Test the model with normalised and without normalisation data by changing the return type
def norm(x):
  #return (x - dframe_describe['mean']) / dframe_describe['std']
  return (x)

normed_data = norm(dframe)
normed_data.head()
  

In [0]:
normed_data_describe = normed_data.describe().transpose()
normed_data_describe.head()

#### Seperate the features and the labels from the dataset

In [0]:
features = normed_data.iloc[:,:-1]
labels =  normed_data['sales']

#### Split the data as test and train in 80:20 ratio

In [0]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

#### Build the dense layer model

In [0]:
def build_model(len_of_input):
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation=tf.nn.relu, input_shape=[len_of_input]),
    tf.keras.layers.Dense(3, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
  ])

  #Learning rate 0.001
  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

In [0]:
len_of_input = len(X_train.keys())
model = build_model(len_of_input)

#### Fit the model on features and lables

In [0]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')


#history = model.fit(X_train, y_train, batch_size=1, epochs=100, validation_split = 0.2,verbose = 0,callbacks=[PrintDot()])
history = model.fit(X_train, y_train, batch_size=1, epochs=200, validation_split = 0.2,verbose = 1)

#### Plot the graph between No.of Epcho's and the Loss function and also the MAE and MSE of training and validation dataset.

In [0]:
import matplotlib.pyplot as plt
plt.xlabel("Epcho")
plt.ylabel("Loss")
plt.plot(history.history['loss'])

In [0]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error Sales')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error Sales')
  plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()
  
plot_history(history)

#### Calculate Loss, MAE and MSE of the model

In [0]:
loss, mae, mse = model.evaluate(X_test, y_test, verbose=0)

print("Testing Loss: {} MAE : {} MSE : {} ".format(loss,mae,mse))

#### Do the prediction on test data

In [0]:
y_pred_dense = model.predict(X_test).flatten()

In [0]:
plt.scatter(y_test, y_pred_dense)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])
plt.show()

#### Observation:

From the above graph b/w epchos and loss we can observe that loss remaining almost constant after 60 epchos.Which means loss is no decreasing gradually with the epchos. But we are still continuing our traning from 60 to 100 epchos for negliable change in loss which is a wastage of CPU cycles.

This kind of scenario has huge impact on CPU when dealing with large and complex dataset.To avoid this kind of problem we can use "Early Stopping" technique of tensflow, which will stop the training if the loss is not changing.





#### Build the model with Early stopping

In [0]:
model = build_model(len_of_input)

# The patience parameter is the amount of epochs to check for improvement
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(X_train, y_train, batch_size=1, epochs=200, validation_split = 0.2,verbose = 1,callbacks=[early_stop])


In [0]:
plt.xlabel("Epcho")
plt.ylabel("Loss")
plt.plot(history.history['loss'])

plot_history(history)

In [0]:
loss, mae, mse = model.evaluate(X_test, y_test, verbose=0)

print("Testing Loss: {} MAE : {} MSE : {} ".format(loss,mae,mse))

#### Comparsion of Model with and Without Early stopping (Below values might change for each execution)

Model parameters without Early Stopping: 
Epchos : 100/100, Testing Loss: 0.01831004787236452, MAE : 0.10798074305057526, MSE : 0.018310047686100006 

Model parameters with Early Stopping:
Epchos : 38/100, Testing Loss: 0.03774606734514237, MAE : 0.15286685526371002, MSE : 0.037746064364910126 

As you can observer early stopping did just 38 epchos to converge the model saving the CPU cycles with almost the same loss, MAE and MSE

#### Do the prediction on test data

In [0]:
y_pred_dense_earlystopping = model.predict(X_test).flatten()


In [0]:
plt.scatter(y_test, y_pred_dense_earlystopping)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])
plt.show()

#### Prediction Using Linear Regression Model

In [0]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()  
regressor.fit(X_train, y_train)

In [0]:
#To retrieve the intercept:
print(regressor.intercept_)
#For retrieving the slope:
print(regressor.coef_)

In [0]:
y_pred_linear = regressor.predict(X_test)


In [0]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_linear})
df

In [0]:
from sklearn import metrics
print ("Linear Regression :")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_linear))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_linear))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_linear)))
print("\n")
print ("Tensorflow DNN :")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_dense))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_dense))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_dense)))
print("\n")
print ("Tensorflow DNN with Early Stopping :")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_dense_earlystopping))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_dense_earlystopping))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_dense_earlystopping)))

#### Observation:

Model with RMSE < 10% of the mean of sales value(14.0225 / 10 = 1.4) is acceptable.

So from above observations we will use tensorflow dense model without early stopping model for our deployment